# Operaciones Básicas

Esta lección cubrirá algunas operaciones básicas con DataFrames de Spark.

Vamos a trabajar con algunos datos de aerolíneas de una muestra pequeña.

In [ ]:
# display() es una función específica de Databricks para visualizar datos de forma interactiva
# dbutils.fs.ls() lista los archivos y directorios en el sistema de archivos de Databricks
# Exploramos qué archivos hay disponibles en el directorio especificado
display(dbutils.fs.ls("/databricks-datasets/asa/small"))

In [ ]:
# Leemos un archivo CSV desde el sistema de archivos de Databricks
# inferSchema=True: Spark analizará los datos para detectar automáticamente los tipos de dato
# header=True: indica que la primera fila del CSV contiene los nombres de las columnas
# Esto es más conveniente pero menos eficiente que especificar el esquema manualmente
df = spark.read.csv("dbfs:/databricks-datasets/asa/small/small.csv", inferSchema=True, header=True)

In [ ]:
# Imprimimos el esquema del DataFrame para entender su estructura
# Esto nos muestra todas las columnas, sus tipos de datos y si aceptan valores nulos
# Vemos que tenemos 29 columnas relacionadas con datos de vuelos
df.printSchema()

In [ ]:
# take(n) devuelve las primeras n filas del DataFrame como una lista de objetos Row
# display() muestra estos datos en un formato visual interactivo de Databricks
# Esto nos permite inspeccionar rápidamente algunos ejemplos de los datos
display(df.take(5))

## Filtrando Datos

Una parte importante del trabajo con DataFrames es la capacidad de filtrar rápidamente datos basados en condiciones. Los DataFrames de Spark están construidos sobre la plataforma Spark SQL, lo que significa que si ya conoces SQL, puedes obtener rápida y fácilmente esos datos usando comandos SQL, o usando los métodos de DataFrame (que es en lo que nos enfocamos en este curso).

In [ ]:
# filter() permite filtrar filas basándose en una condición
# Aquí usamos sintaxis SQL: seleccionamos solo vuelos donde ActualElapsedTime < 120 minutos
# El resultado es un nuevo DataFrame con solo las filas que cumplen la condición
# display() muestra estos datos filtrados en formato visual
display(df.filter("ActualElapsedTime<120"))

In [ ]:
# Encadenamos operaciones: primero filtramos, luego seleccionamos una columna específica
# filter() reduce las filas según la condición
# select() selecciona solo la columna "DepTime" del resultado filtrado
# show() muestra las primeras 20 filas en formato tabla ASCII
df.filter("ActualElapsedTime<120").select("DepTime").show()

In [ ]:
# Sintaxis alternativa usando notación tipo pandas con corchetes
# df.ActualElapsedTime accede a la columna sin comillas (requiere que el nombre sea válido en Python)
# Esta sintaxis es útil cuando la columna no está distribuida o cuando prefieres sintaxis Python
# El resultado es idéntico al ejemplo anterior
df[df.ActualElapsedTime<120].select("DepTime").show()

In [ ]:
# Combinamos filtro con selección de múltiples columnas
# select() puede recibir una lista de nombres de columnas
# Aquí seleccionamos "DayOfWeek" y "DayOfMonth" de los vuelos filtrados
# Útil para análisis donde necesitamos ver varias columnas relacionadas
df.filter("ActualElapsedTime<120").select(["DayOfWeek", "DayOfMonth"]).show()

Usar operadores de comparación normales de Python es otra forma de hacer esto, se verán muy similares a los operadores SQL, excepto que necesitas asegurarte de que estás llamando a la columna completa dentro del dataframe, usando el formato: df["nombre_columna"]

Veamos algunos ejemplos:

In [ ]:
# Usamos operadores de comparación de Python directamente sobre columnas
# df["ActualElapsedTime"] < 200 crea una expresión booleana
# filter() evalúa esta expresión para cada fila y mantiene solo las que son True
# Esta sintaxis es más "pythónica" que la sintaxis SQL con comillas
display(df.filter(df["ActualElapsedTime"] < 200))

In [ ]:
# EJEMPLO DE ERROR: intentamos usar 'and' de Python con columnas de Spark
# Esto falla porque 'and' es un operador de Python, no de Spark
# En PySpark debemos usar operadores bitwise: & para AND, | para OR, ~ para NOT
# Este código está comentado porque genera un ValueError
df.filter(df["DayOfWeek"]<5 and df["DayOfMonth"]> 15).show()

In [ ]:
# FORMA CORRECTA: usamos & en lugar de 'and' para combinar condiciones
# Los paréntesis son OBLIGATORIOS alrededor de cada condición
# (df["DayOfWeek"]<5) filtra días laborables (lunes=1 a jueves=4)
# (df["DayOfMonth"]> 15) filtra la segunda mitad del mes
# & combina ambas condiciones: ambas deben ser True
display(df.filter((df["DayOfWeek"]<5) & (df["DayOfMonth"]> 15)))

In [ ]:
# Filtramos por una sola condición y seleccionamos múltiples columnas
# DayOfWeek < 5 significa lunes (1) a jueves (4), excluyendo viernes-domingo
# select() con lista nos da solo las columnas de interés del DataFrame filtrado
# Útil para análisis enfocados en variables específicas
df.filter(df["DayOfWeek"] < 5).select(["ActualElapsedTime", "TailNum", "AirTime"]).show()

In [ ]:
# Ejemplo combinado: filtro con dos condiciones y selección limitada
# & requiere que AMBAS condiciones sean verdaderas
# take(5) limita el resultado a solo 5 filas (más eficiente que .show(5))
# display() muestra estas 5 filas en formato visual interactivo
display(df.filter((df["DayOfWeek"] < 5) & (df["DayOfMonth"] > 20)).take(5))

In [ ]:
# Operador | (pipe) representa OR lógico en PySpark
# Una fila se incluye si CUALQUIERA de las condiciones es verdadera
# DayOfWeek < 5 (días laborables) O DayOfMonth > 20 (final del mes)
# Esto incluirá más filas que usar & (AND)
display(df.filter((df["DayOfWeek"] < 5) |(df["DayOfMonth"] > 20)).take(5))

In [ ]:
# Operador ~ (tilde) representa NOT lógico en PySpark
# ~(df["DayOfMonth"] < 20) es equivalente a df["DayOfMonth"] >= 20
# Combinamos: días laborables (DayOfWeek < 5) Y día 20 o posterior del mes
# El NOT es útil cuando la condición negativa es más natural de expresar
display(df.filter((df["DayOfWeek"] < 5) & ~(df["DayOfMonth"] < 20)).take(5))

In [ ]:
# Filtro por igualdad exacta usando ==
# DayOfWeek == 2 selecciona solo los martes (donde 1=lunes, 2=martes, etc.)
# take(2) devuelve solo las primeras 2 filas que cumplen la condición
# Útil para análisis de patrones específicos de días de la semana
display(df.filter(df["DayOfWeek"] == 2).take(2))

In [ ]:
# collect() trae TODAS las filas filtradas a la memoria del driver como lista
# CUIDADO: usar collect() con grandes datasets puede causar problemas de memoria
# Solo usar collect() cuando estés seguro de que el resultado es pequeño
# display() muestra todas las filas del día 3 del mes
display(df.filter(df["DayOfMonth"] ==3).collect())

In [ ]:
# Filtramos vuelos del miércoles (DayOfWeek == 3) y guardamos el resultado
# collect() devuelve una lista de objetos Row en Python
# Almacenamos esta lista en 'result' para procesamiento posterior
# Es importante distinguir: result es una lista de Python, no un DataFrame
result = df.filter(df["DayOfWeek"] == 3).collect()

In [ ]:
# Verificamos el tipo del primer elemento de la lista
# result[0] es un objeto Row de PySpark
# Row es una estructura que representa una fila con acceso por nombre de columna
# Similar a un diccionario pero optimizado para Spark
type(result[0])

In [ ]:
# Guardamos la primera fila (Row) en la variable 'row' para análisis detallado
# Esto nos permite trabajar con una fila individual de forma más conveniente
# Podemos acceder a los valores de la fila usando row['nombre_columna'] o row.nombre_columna
row = result[0]

In [ ]:
# asDict() convierte el objeto Row en un diccionario estándar de Python
# Las claves del diccionario son los nombres de las columnas
# Los valores son los datos correspondientes de esa fila
# Útil para serialización, debugging, o cuando necesitas trabajar con diccionarios
row.asDict()

In [ ]:
# Iteramos sobre todos los valores de la primera fila
# Un objeto Row es iterable: recorre los valores en el orden de las columnas
# Cada 'item' es el valor de una columna en esa fila específica
# Imprimimos cada valor en una línea separada para inspección detallada
for item in result[0]:
    print(item)

¡Eso es todo por ahora! ¡Buen trabajo!